# AIC23

Load dataset and launch app

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
import numpy as np

dataset = fo.Dataset.from_images_dir("./data/keyframes", name=None, tags=None, recursive=True)
session = fo.launch_app(dataset, desktop=False)
session.wait()

In [ ]:
import os
import csv
import pandas as pd

info = dict()
for file in os.scandir("map-keyframes"):
    if file.name[0] == '.':
        continue
    videoId = file.name.split(".")[0]
    data = pd.read_csv(file.path)
    info[videoId] = data.to_dict()
    print(info[videoId]['n'])

In [19]:
for sample in dataset:
    videoId, frameId = sample["filepath"].split("/")[-2:]
    videoId = videoId.split(".")[0]
    frameId = int(frameId.split(".")[0])
    sample["videoId"] = videoId
    sample["n"] = str(info[videoId]['n'][frameId-1])
    sample["frameId"] = str(info[videoId]['frame_idx'][frameId-1])
    sample["pts_time"] = str(info[videoId]['pts_time'][frameId-1])
    sample.save()

### Embeddings of videos are provided 

Compute embeddings (if not provided)

In [ ]:
model = foz.load_zoo_model("clip-vit-base32-torch")
embeddings = dataset.compute_embeddings(model)
with open("./embeddings/keyframes.npy", "wb") as f:
    np.save(f, embeddings);

Load embeddings

In [5]:
embeddings = np.load("./embeddings/keyframes.npy")

Compute visualization using embeddings

In [ ]:
results = fob.compute_visualization(
    dataset, 
    embeddings=embeddings, 
    seed=51, 
    brain_key="img_viz"
)

Compute similarity (used to sort by similarity or query by text)

In [6]:
image_index = fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch", 
    embeddings=embeddings,       
    brain_key="img_sim",
)